# Error-correcting codes for the binary symmetric channel

In this notebook, I try out the Hamming code and test it for its bit error rate.
Furthermore, I try to generate other encodings with the same rate of information and block length to see how the Hamming code compares to them.

In [1]:
import random

In this first part, I define some help functions to get the calculations done.

### 1) Get all possible combinations of n bits, ordered by their binary value

In [2]:
def get_possible_combinations (n):
    if n==1:
        # basic case
        return (['0', '1'])
    else:
        # recursive step; 
        combinations = get_possible_combinations(n-1)
        # for each combination of length n-1, take one with a 1 on front
        # and one with a zero
        return(['0' + combination for combination in combinations]
               + ['1' + combination for combination in combinations])

For example, the ordered possible combinations of 3 bits are:

In [3]:
get_possible_combinations(3)

['000', '001', '010', '011', '100', '101', '110', '111']

Note that the index in the list of each element is its binary value.

### 2) Random binary encoding (with added bits): for each input gives a valid output

An encoding is a mapping input-output for all possible inputs. </br>
In its explicit form, it's simply a list of outputs (may not scale well). </br>
Since the input is assumed to be complete (all possible combinations) and ordered (as above), having the encodings (redundant bits) is enough to know the function: the mapping is done through indexing from the binary number represented by the input.
In this case, I only consider encodings for which the first bits remain the same as the input and some additional "check bits" (e.g. parity bits) are added.

In [4]:
def get_random_encoding (inputs, outputs):
    return ([outputs[random.randrange(0,len(outputs))] for i in inputs])

For example a random encoding of 3 possible inputs, with 4 possible outputs, returns a randomly selected output for each possible input.

In [5]:
get_random_encoding([1,2,3], ['01', '00', '10', '11'])

['01', '00', '11']

In its matrix form, the encoding would require less memory (less values to be stored), but more calculations to find the nearest neighbor (matrix multiplications instead of list lookups) and restriction to linear cases.

### 3) Finding the closest message to the received one (r)

The encodings are all ordered in a list. </br>
A message is considered the closest to a given one if the amount of bits of difference is minimal across all possibilities.</br>
Because of the chosen implementation, if more such messages are possibile, the one with the leftmost bit of difference to the received message is chosen.

In [6]:
# find the valid message (= with valid encoding) closest to the inputted one
def find_closest (r, encoding, block_length):
    
    # if the received message r is valid (the first <block_length> bits give an encoding equal
    # to the last bits), the same encoding is returned
    if encoding[int(r[:block_length],2)] == r[block_length:]:
        return r
    
    # else, systematically look at all inputs;
    # start by changing 1 single bit and seeing if the message is valid;
    # when all possible messages with <n> changes are tested and invalid, start testing with <n+1> bits
    
    possibilities = [r]
    # continue loop until valid neighbor found
    while (True):
        for possibility in possibilities:
            
            # systematically change 1 bit
            for i in range(len(possibility)):
                
                # change bit i from an already unsuccessfully tested possibility
                new_string = possibility[:i] + str(int(not bool(int(possibility[i])))) + (
                            possibility[i+1:] if i!=(len(possibility)-1) else '')
                    
                # increase list from the right side, so that the last possibility added will be the last tested (FIFO)
                possibilities.append(new_string)
                index = int(new_string[:block_length], 2)
                check = encoding[index]
                
                # test if valid input-output message
                if (check==new_string[block_length:]):
                    
                    # if valid, return as closest valid message
                    return new_string     

Example of how the method works

In [7]:
print(find_closest ('0110', ['10', '11', '11', '11'], 2))

0010


### 4) Methods for systematical mathematical evaluation of all cases

The probability of a case of <nb_flips> flips with message length <message_len> and flipping probability for each bit <f>.
Binary distribution.

In [8]:
def flipping_probability (message_len, f, nb_flips):
    return ((1-f)**(message_len-nb_flips)) * (f**nb_flips)

Get a list of all flipping probabilities, ordered from 0 flips to <message_len> flips

In [9]:
def list_flipping_probabilities (message_len, f):
    return [flipping_probability(message_len, f, nb_flips) for nb_flips in range(message_len+1)]

Get all possible sublists of a list.

In [10]:
def sublists (lst):
    if len(lst)==0:
        return [[]]
    else:
        return [[lst[0]] + sub for sub in sublists(lst[1:])] + [sub for sub in sublists(lst[1:])]
    
# example working
sublists([1,2,3])

[[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]

### 5) Testing methods

Count the number of different bits between two messages, considering the actual sensor message s (of length <block_len>) only.

In [11]:
def bit_errors (block_len, t, error_correcting):
    n = 0
    for i in range(block_len):
        if error_correcting[i] != t[i]:
            n+=1
    return n

Test a given encoding (in the form of a list) with probability of bit flip f. </br>
Tests for all possible messages + encodings + noise vectors and weights according to probability of the combination: hence, the result should be exact up to approximations of the calculations.
It makes use of the closest possibility method as defined above to pick the (error-corrected)-corresponding message, and compares it to the sent one.

In [12]:
def test_encoding (block_length, encoding, f):
    
    probabilities = list_flipping_probabilities (block_length + len(encoding[0]), f)
    
    # initialize error rates
    block_error_rate = 0
    bit_error_rate = 0
    
    # test for all flip_patterns (possible errors)
    for flip_pattern in sublists(list(range(block_length+len(encoding[0])))):
        
        #initialize counter
        n = 0
        # initialize block error rates
        pattern_block_error_rate = 0
        pattern_bit_error_rate = 0
        # initialize input to all 0s
        s = '0'*block_length
        
        while (True):
            n+=1
            error = 0
            # add encoding for actual input
            redundancy = encoding[int(s,2)]
            t = s + redundancy
            # received message
            r = ''
            # flip bits in received message (from transmitted one) according to given pattern
            for index in range(len(t)):
                r = r + (str(int(not bool(int(t[index])))) if index in flip_pattern else t[index])
                
            # if r == t, correct decoding; else, look at closest message
            if r != t:
                error_correcting = find_closest (r, encoding, block_length)
                # if closest message (error-corrected decoding different from the transmitted one), error
                if (error_correcting != t):
                    # update error rates
                    pattern_block_error_rate += 1
                    pattern_bit_error_rate += bit_errors(block_length, t, error_correcting)
            
            # increase input
            s = '{0:b}'.format(int(s,2)+1)
            diff = block_length - len(s)
            
            # finished possibilities for the actual cycle (pattern) if longer pattern
            # (all 1's exceeded); hence, exit the loop
            if (diff < 0):
                break
            # reset length (add required 0's in front that were eliminated by the int operations)
            s = '0'*diff + s
            
        # update error rates to probability-adjusted pattern error rates
        block_error_rate += probabilities[len(flip_pattern)] * pattern_block_error_rate
        bit_error_rate += probabilities[len(flip_pattern)] * pattern_bit_error_rate / (block_length)
        
    # since tested for all possible inputs, to get the mean divide by the number of possible inputs
    # (assumed to be equiprobable)
    nb_possible_inputs = len(encoding)
    block_error_rate /= nb_possible_inputs
    bit_error_rate /= nb_possible_inputs
        
    return (block_error_rate, bit_error_rate)

Can be used if the exact test encoding takes too long. </br>
For all possible inputs, tests with some random generated noise vectors.

In [13]:
def test_encoding_approximate (block_length, encoding, f, cycles = 100):
    n = 0
    block_error_rate = 0
    bit_error_rate = 0
    
    # test on complete input space for <cycles> cycles
    for i in range(cycles):
        s = '0'*block_length
        
        # test on all possible inputs
        while (True):
            error = 0
            bit_error = 0
            redundancy = encoding[int(s,2)]
            t = s + redundancy
            r = ''
            
            # for each bit, flip with probability f
            for c in t:
                r = r + str(int((not bool(int(c)) if random.random() < f else bool(int(c)))))
                
            # no checking if not changed
            if t != r:
                error_correcting = find_closest (r, encoding, block_length)
                if (error_correcting != t):
                    error = 1
                    bit_error = bit_errors (block_length, t, error_correcting)
            n+=1
            block_error_rate = ((n-1)/n)*block_error_rate + error*(1/n)
            bit_error_rate = ((n-1)/n)*bit_error_rate + bit_error*(1/n)/4
            s = '{0:b}'.format(int(s,2)+1)
            diff = block_length - len(s)
            if (diff < 0):
                break
            s = '0'*diff + s
    return (block_error_rate, bit_error_rate)

Generate a random encoding and test it using the above method.

In [14]:
def test_random_encodings (inputs, outputs, block_length, added_length, f, nb_iterations, approximate=False):
    
    def funct(results, index):
        best_bit_error_rate = 1
        best_encoding = []
        for i in range(nb_iterations):
            encoding = get_random_encoding (inputs, outputs)
            if not approximate:
                block_error_rate, bit_error_rate = test_encoding (block_length, encoding, f)
            else:
                block_error_rate, bit_error_rate = test_encoding_approximate (block_length, encoding, f)
            if bit_error_rate < best_bit_error_rate:
                best_bit_error_rate = bit_error_rate
                best_encoding = encoding
        results[index] = (best_bit_error_rate, best_encoding)
        return
        
    return funct

Test several random encodings and print out the best one.

In [15]:
import numpy as np
import multiprocessing

def best_random_encoder (block_length, added_length, f, nb_encodings, approximate=False):
    nb_processes = 3
    inputs = get_possible_combinations (block_length)
    outputs = get_possible_combinations (added_length)
    nb_iterations = nb_encodings//nb_processes
    testing_function = test_random_encodings (inputs, outputs, block_length, added_length, f, nb_iterations, approximate)
    
    processes = []
    manager = multiprocessing.Manager()
    results = manager.dict()
    for i in range(nb_processes):
        p = multiprocessing.Process(target=testing_function, args=(results,i))
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
        p.terminate()
        
    encodings = [results[data][1] for data in results]
    error_rates = [results[data][0] for data in results]
    index = np.argmin(error_rates)
    print (results[index])

## Test some random-generated encodings and print the best one (7-4 encoders)

In [16]:
best_random_encoder (4, 3, 0.1, 100000)

(0.08080680000000003, ['011', '000', '101', '000', '001', '110', '101', '110', '111', '111', '010', '100', '100', '001', '011', '001'])


## Test Hamming encoder

In [17]:
hamming_encoding = [
    '000',
    '011',
    '111',
    '100',
    '110',
    '101',
    '001',
    '010',
    '101',
    '110',
    '010',
    '001',
    '011',
    '000',
    '100',
    '111',
]
block_rate, bit_rate = test_encoding (4, hamming_encoding, 0.1)
print(block_rate, bit_rate)

0.1496944000000001 0.06688000000000002


In [18]:
print (test_encoding_approximate(4, hamming_encoding, 0.1))

(0.15249999999999983, 0.06828124999999988)


Compare with estimated values for the Hamming code: 21f**2 = 0.21, 9f**2 = 0.09:
even better than estimated!

In [19]:
# the ratio of 3/7 (parity) holds:
bit_rate*7/3 - block_rate

0.006358933333333289